In [ ]:
# https://ppiazi.tistory.com/entry/python-cyclomatic-complexity-%EB%AA%A8%EB%93%88-lizard

In [ ]:
# https://github.com/terryyin/lizard

In [ ]:
# pip install cognitive_complexity

In [1]:
import sys, os
p = os.path.abspath('..')
pp = os.path.abspath('../..')
sys.path.insert(1, p)
sys.path.insert(1, pp)

In [2]:
#customized preprocessing pkg
import preprocess as pp
import pandas as pd
import numpy as np
import pickle
import ast
import pprint
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import config.config as conf
# https://joaorafaelm.github.io/notebook/python/2021/04/10/cognitive-complexity.html
# complexity_python_df_2023_2324

In [9]:
input_year = '2023'
input_pl = 'python'
year_range = '2324'


### 소스코드 complexity 계산

In [ ]:
# 파일 불러오기
with open(f'{conf.DATA_PATH}/data/complexity_{input_pl}_df_{input_year}_{year_range}.pkl', 'rb') as f:
    complexity_python_df = pickle.load(f)

In [5]:
complexity_python_df['cdate'].max()

'2023-11-30'

In [6]:
htmlp = pp.HTMLParser()
codep = pp.CodeSectionParser()

In [7]:
complexity_python_df['c_h_body'] = complexity_python_df['body'].apply(lambda x : htmlp.clean_html_str(x))

In [8]:
complexity_python_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120673 entries, 0 to 120672
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        120673 non-null  int64 
 1   cdate     120673 non-null  object
 2   title     120673 non-null  object
 3   body      120673 non-null  object
 4   c_h_body  120673 non-null  object
dtypes: int64(1), object(4)
memory usage: 4.6+ MB


In [9]:
complexity_python_df['c_body'] = complexity_python_df['body'].apply(lambda x : codep(x))

In [10]:
complexity_python_df

,id,cdate,title,body,c_h_body,c_body
0,75072182,2023-01-10,How can I access all PyMol commands through a ...,<p>Currently trying to use a Python script to ...,Currently trying to use a Python script to aut...,"{'code_sections': [{'off_beg': 307, 'off_end':..."
1,75072191,2023-01-10,Why does my string doesn't get modified to one...,"<p>I have a number 'n' (n=60, for example), my...","I have a number 'n' (n=60, for example), my ta...","{'code_sections': [{'off_beg': 669, 'off_end':..."
2,75072200,2023-01-10,How to create a column based on the value of t...,<p>I have the following dataframe</p>\n<pre><c...,I have the following dataframe\n type_x Ra...,"{'code_sections': [{'off_beg': 49, 'off_end': ..."
3,75072207,2023-01-10,Why does read_csv give me a timezone warning?,<p>I try reading a CSV file using pandas and g...,I try reading a CSV file using pandas and get ...,"{'code_sections': [{'off_beg': 95, 'off_end': ..."
4,75072229,2023-01-10,How to use hash function in Python3 to transfo...,<p>I have a large number of different sentence...,I have a large number of different sentences w...,{'code_sections': []}
...,...,...,...,...,...,...
120668,77581957,2023-11-30,Seaborn's histplot doesn't match Matplotlib's ...,"<p>I'm probably making a stupid mistake, but h...","I'm probably making a stupid mistake, but here...",{'code_sections': []}
120669,77581984,2023-11-30,Removing Rectangular Surfaces in 3D Matplotlib...,<p>I am trying to reproduce a 3D plot of a hel...,I am trying to reproduce a 3D plot of a helica...,"{'code_sections': [{'off_beg': 329, 'off_end':..."
120670,77582066,2023-11-30,Efficent way to to get values from one pandas ...,<p>I have two data sets. One contains 3 ish mi...,I have two data sets. One contains 3 ish milli...,{'code_sections': []}
120671,77582132,2023-11-30,Error when trying to import data from a CSV fi...,<p>I am trying to insert a CSV file that is st...,I am trying to insert a CSV file that is store...,"{'code_sections': [{'off_beg': 403, 'off_end':..."


In [11]:
complexity_python_src = complexity_python_df[['id', 'c_body']]

In [12]:
complexity_python_src['code_len'] = complexity_python_src['c_body'].apply(lambda x: len(x['code_sections']))

/tmp/ipykernel_2691809/2692464613.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complexity_python_src['code_len'] = complexity_python_src['c_body'].apply(lambda x: len(x['code_sections']))


In [13]:
complexity_python_src = complexity_python_src[complexity_python_src['code_len']>0]

In [14]:
complexity_python_src.head(5)

,id,c_body,code_len
0,75072182,"{'code_sections': [{'off_beg': 307, 'off_end':...",1
1,75072191,"{'code_sections': [{'off_beg': 669, 'off_end':...",1
2,75072200,"{'code_sections': [{'off_beg': 49, 'off_end': ...",3
3,75072207,"{'code_sections': [{'off_beg': 95, 'off_end': ...",2
5,75072241,"{'code_sections': [{'off_beg': 321, 'off_end':...",1


In [17]:
src_df = pd.DataFrame(columns = ['id', 'src'])
def code_explode(row) : 
    global src_df
    row['id']
    src_arr = []
    for i in range(row['code_len']):
        tmp_src_arr = []
        tmp_src_arr.append(row['id'])
        tmp_src_arr.append(row['c_body']['code_sections'][i]['span_str'])

         # append 대신 concat 사용
        new_row = pd.Series(tmp_src_arr, index=src_df.columns)
        src_df = pd.concat([src_df, new_row.to_frame().T], ignore_index=True)
    
    return True
        

In [18]:
# bert_src_ex = bert_src[['q_id', 'q_prep_text']].apply(pd.Series.explode)
complexity_python_src.apply(code_explode, axis = 1)

0         True
1         True
2         True
3         True
5         True
          ... 
120665    True
120666    True
120669    True
120671    True
120672    True
Length: 87952, dtype: bool

In [19]:
src_df.reset_index(inplace=True)

In [20]:
src_df.head()

,index,id,src
0,0,75072182,"from pymol import cmd\n\npymol.cmd.do(""fab APA..."
1,1,75072191,class Solution(object):\n\n # (60)\n\n d...
2,2,75072200,type_x Range myValname\n0 g1 0.48 ...
3,3,75072200,type_x Range myValname newCol\n0 g1 ...
4,4,75072200,df3['newCol'] = df3.groupby('myValname').rank...


In [ ]:
for i in range(src_df.shape[0]):
    idx = src_df.iloc[i, 0]
    qid = src_df.iloc[i, 1]
    src = src_df.iloc[i, 2]
    file_nm = str(idx)+'_' + str(qid)+'.py'
    with open(f'{conf.DATA_PATH}/data/src/{input_year}_src_{year_range}/{file_nm}', 'wb') as f:
        f.write(src.encode())


In [ ]:
# https://github.com/rohaquinlop/complexipy

In [ ]:
# pip install complexipy

In [ ]:
# !complexipy /home/mghan/sopjt/git/data/src/3_74963438.py -l file -o

In [ ]:
# cyclomatic은 모든 언어 가능 but cognitive는 언어별로 따로 분석해야함
# 하려면 소나큐브 사야 함 
# pyhon 기준으로는 조금 삽질하면 할 수 있을 듯.... 해보고 유의미하면 소나큐브 고?


### complexity 결과 확인

In [4]:
path = f'{conf.DATA_PATH}/data/src/{input_year}_excel_{year_range}/'
file_list = os.listdir(path)
file_list_csv = [file for file in file_list if file.endswith(".csv")]

# print ("file_list_csv: {}".format(file_list_csv))

In [5]:
calc_com_df = pd.DataFrame(columns = ['Path', 'File Name', 'Cognitive Complexity'])
for file_nm in file_list_csv :
    tmp = pd.read_csv(f'{path}/{file_nm}')
    calc_com_df = pd.concat([calc_com_df, tmp], axis = 0)

In [6]:
calc_com_df.head()

,Path,File Name,Cognitive Complexity
0,38173_77388248.py,38173_77388248.py,0
0,41365_77332287.py,41365_77332287.py,1
0,80506_76638091.py,80506_76638091.py,0
0,73019_76749980.py,73019_76749980.py,0
0,117423_76013942.py,117423_76013942.py,0


In [10]:
# 전체 데이터셋 
with open(f'{conf.DATA_PATH}/data/calc_com_df_{input_year}_{input_pl}_{year_range}.pkl', 'wb') as f:
    pickle.dump(calc_com_df, f)

In [ ]:
calc_com_df.head()

In [ ]:
calc_com_df.reset_index(drop=True, inplace=True)

In [ ]:
calc_com_df.head()

In [ ]:
calc_com_df['index'] = calc_com_df['Path'].apply(lambda x : x.split('_')[0])
# calc_com_df['Path'].apply(lambda x : [  for i in x.split('_')])

In [ ]:
calc_com_df['id'] = calc_com_df['Path'].apply(lambda x : x.split('_')[1].split('.')[0])

In [ ]:
calc_com_df.sort_values(by = ['index', 'id'])

In [ ]:
calc_com_df[['index', 'id', 'Cognitive Complexity']] = calc_com_df[['index', 'id', 'Cognitive Complexity']].astype('int')

In [ ]:
src_df[['index', 'id']] = src_df[['index', 'id']].astype('int')

In [ ]:
calc_com_df.info()

In [ ]:
calc_com_df.sort_values(by = ['index', 'id'])

In [ ]:
calc_com_df.info()

In [ ]:
src_df.info()

In [ ]:
calc_com_df = pd.merge(src_df,  calc_com_df, how = 'left', on = ['index', 'id'])

In [ ]:
calc_com_df = pd.merge(complexity_python_df[['id', 'cdate']], calc_com_df, on = 'id')

In [ ]:
calc_com_df.head()

In [ ]:
# 일자별로 아예 소스가 수행되지 않는 건 
calc_com_df_back = calc_com_df.copy()
calc_ed_com_df = calc_com_df[~calc_com_df['Path'].isna()]

In [ ]:
calc_ed_com_df = calc_ed_com_df.sort_values(by = ['cdate', 'id', 'Cognitive Complexity'], ascending = [True, True, False]).groupby('id').head(1)

In [ ]:
calc_ed_com_df.head()

In [ ]:
calc_ed_com_df = calc_ed_com_df.groupby('cdate').mean()[['Cognitive Complexity']].reset_index()

In [ ]:
calc_ed_com_df.head()

In [ ]:
calc_ed_com_box_df = calc_ed_com_df.copy()

In [ ]:
calc_ed_com_box_df['yyyymm'] = calc_ed_com_box_df['cdate'].str[:7]

In [ ]:
ym_list = calc_ed_com_box_df['yyyymm'].unique()

In [ ]:
[calc_ed_com_box_df[calc_ed_com_box_df['yyyymm']==x]  for x in ym_list]

In [ ]:
sharey = True ## 또는 sharey=False
sharex = True ## 또는 sharex=False

fig, axs = plt.subplots(1, 2, figsize = (20, 5), sharey=sharey, sharex=sharex)
# plt.xticks(rotation=45)
# major_xticks = ['2013-11-30','2014-11-30','2015-11-30','2016-11-30','2017-11-30','2018-11-30','2019-11-30','2020-11-30'
#                 ,'2021-11-30', '2022-11-30', '2023-11-30']
# yticks = np.arange(200, 2000, 400)
axs[0].bar(calc_ed_com_df['cdate']
        ,  calc_ed_com_df['Cognitive Complexity']
        ,  color = 'skyblue'
        # ,  edgecolor = 'black'
      ) # hist type

axs[0].set_title('Number of Questions over time', fontsize = 15)
axs[0].set_xlabel('Date', fontsize = 15)
axs[0].set_ylabel('# of Q', fontsize = 15)
# axs[0].set_xticks(major_xticks)
# axs[0].set_yticks(yticks)
# axs[0].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[0].tick_params(axis='x', rotation=45)
# axs[0].yticks(yt)

axs[0].bar(calc_ed_com_df['cdate']
        ,  calc_ed_com_df['Cognitive Complexity']
        ,  color = 'skyblue'
        # ,  edgecolor = 'black'
      ) # hist type

axs[1].set_title('Number of Answers over time', fontsize = 15)
axs[1].set_xlabel('Date', fontsize = 15)
axs[1].set_ylabel('# of A', fontsize = 15)
# axs[1].set_xticks(major_xticks)
# axs[1].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[1].tick_params(axis='x', rotation=45)

plt.show();


In [ ]:
sharey = True ## 또는 sharey=False
sharex = True ## 또는 sharex=False

fig, axs = plt.subplots(1, 2, figsize = (20, 5), sharey=False, sharex=False)
# plt.xticks(rotation=45)
major_xticks = ['2022-01-30','2022-02-28','2022-03-30','2022-04-30','2022-05-30','2022-06-30','2022-07-30','2022-08-30','2022-09-30','2022-10-30','2022-11-30','2022-12-30',
                '2023-01-30','2023-02-28','2023-03-30','2023-04-30','2023-05-30','2023-06-30','2023-07-30','2023-08-30','2023-09-30','2023-10-30','2023-11-30','2023-12-30']
axs1_xticks = ym_list
axs1_xticks = np.insert(axs1_xticks, 0, '0')  # 인덱스 2 위치에 100을 추가 
# yticks = np.arange(200, 2000, 400)
axs[0].bar(calc_ed_com_df['cdate']
        ,  calc_ed_com_df['Cognitive Complexity']
        ,  color = 'skyblue'
        # ,  edgecolor = 'black'
      ) # hist type

axs[0].set_title('Avg Cognitive Complexity per Day', fontsize = 15)
axs[0].set_xlabel('Date', fontsize = 15)
axs[0].set_ylabel('Avg Cognitive Complexity', fontsize = 15)
axs[0].set_xticks(major_xticks)
# axs[0].set_yticks(yticks)
# axs[0].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[0].tick_params(axis='x', rotation=45)
# axs[0].yticks(yt)

axs[1].boxplot([calc_ed_com_box_df.loc[calc_ed_com_box_df['yyyymm']==x, 'Cognitive Complexity']  for x in ym_list]
        # ,  edgecolor = 'black'
      ) # hist type

axs[1].set_title('Cognitive Complexity per Month', fontsize = 15)
axs[1].set_xlabel('Date', fontsize = 15)
axs[1].set_ylabel('Cognitive Complexity', fontsize = 15)
axs[1].set_xticks(range(len(axs1_xticks)), axs1_xticks)
# axs[1].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[1].tick_params(axis='x', rotation=45)

plt.show();


In [ ]:
ym_list[:11]
# ym_list[12:]

In [ ]:
sharey = True ## 또는 sharey=False
sharex = True ## 또는 sharex=False

fig, axs = plt.subplots(2, 1, figsize = (10, 10), sharey=True, sharex=False)
# plt.xticks(rotation=45)

axs0_xticks = ym_list[:11]
axs0_xticks = np.insert(axs0_xticks, 0, '0')  # 인덱스 2 위치에 100을 추가 

axs1_xticks = ym_list[12:]
axs1_xticks = np.insert(axs1_xticks, 0, '0')  # 인덱스 2 위치에 100을 추가 
yticks = np.arange(0.5, 5, 0.5)

axs[0].boxplot([calc_ed_com_box_df.loc[calc_ed_com_box_df['yyyymm']==x, 'Cognitive Complexity']  for x in ym_list[:11]]
        # ,  edgecolor = 'black'
      ) # hist type

axs[0].set_title('Cognitive Complexity per Month', fontsize = 15)
axs[0].set_xlabel('Date', fontsize = 15)
axs[0].set_ylabel('Cognitive Complexity', fontsize = 15)
axs[0].set_xticks(range(len(axs0_xticks)), axs0_xticks)
axs[0].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[0].tick_params(axis='x', rotation=45)


axs[1].boxplot([calc_ed_com_box_df.loc[calc_ed_com_box_df['yyyymm']==x, 'Cognitive Complexity']  for x in ym_list[12:]]
        # ,  edgecolor = 'black'
      ) # hist type

axs[1].set_title('Cognitive Complexity per Month', fontsize = 15)
axs[1].set_xlabel('Date', fontsize = 15)
axs[1].set_ylabel('Cognitive Complexity', fontsize = 15)
axs[1].set_xticks(range(len(axs1_xticks)), axs1_xticks)
axs[1].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[1].tick_params(axis='x', rotation=45)
plt.tight_layout()    
plt.show();


In [ ]:
len(np.arange(2, 13, 1))

In [ ]:
sharey = True ## 또는 sharey=False
sharex = True ## 또는 sharex=False

fig, axs = plt.subplots(2, 1, figsize = (10, 10), sharey=True, sharex=False)
# plt.xticks(rotation=45)

axs0_xticks = ym_list[:11]
axs0_xticks = np.insert(axs0_xticks, 0, '0')  # 인덱스 2 위치에 100을 추가 

axs1_xticks = ym_list[12:]
axs1_xticks = np.insert(axs1_xticks, 0, '0')  # 인덱스 2 위치에 100을 추가 
yticks = np.arange(0.5, 5, 0.5)


axs[0].violinplot([calc_ed_com_box_df.loc[calc_ed_com_box_df['yyyymm']==x, 'Cognitive Complexity']  for x in ym_list[:11]], positions=np.arange(1, 12, 1))
axs[0].set_title('Cognitive Complexity per Month', fontsize = 15)
axs[0].set_xlabel('Date', fontsize = 15)
axs[0].set_ylabel('Cognitive Complexity', fontsize = 15)
axs[0].set_xticks(range(len(axs0_xticks)), axs0_xticks)
axs[0].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[0].tick_params(axis='x', rotation=45)



axs[1].violinplot([calc_ed_com_box_df.loc[calc_ed_com_box_df['yyyymm']==x, 'Cognitive Complexity']  for x in ym_list[12:]], positions=np.arange(1, 12, 1))
axs[1].set_title('Cognitive Complexity per Month', fontsize = 15)
axs[1].set_xlabel('Date', fontsize = 15)
axs[1].set_ylabel('Cognitive Complexity', fontsize = 15)
axs[1].set_xticks(range(len(axs1_xticks)), axs1_xticks)
axs[1].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[1].tick_params(axis='x', rotation=45)
plt.tight_layout()    
plt.show();
